In [2]:
import cv2
import mediapipe as mp
import numpy as np

# Bring the mediapipe modules
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands()

# Create a black image which will act as a canvas
canvas = np.zeros((480, 640, 3), dtype=np.uint8)

# Function to detect if the index finger is up..blah blah blah..(copy paste)
def is_index_finger_up(hand_landmarks):
    # Assuming the hand_landmarks has only one hand.
    landmarks = hand_landmarks.landmark
    return landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP].y < landmarks[mp_hands.HandLandmark.INDEX_FINGER_PIP].y

# Capture video from the webcam
cap = cv2.VideoCapture(0)

prev_x, prev_y = 0, 0
while cap.isOpened():
    success, image = cap.read()
    if not success:
        break

    # Flip the image horizontally for a later selfie-view display...because initially it will be y-axis rotated
    image = cv2.flip(image, 1)
    # Convert the BGR image to RGB
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Process the image and detect hands
    results = hands.process(rgb_image)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            if is_index_finger_up(hand_landmarks):
                # Get the tip of the index finger
                tip_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image.shape[1])
                tip_y = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image.shape[0])
                if prev_x and prev_y:
                    cv2.line(canvas, (prev_x, prev_y), (tip_x, tip_y), (255, 0, 0), 5)
                prev_x, prev_y = tip_x, tip_y
            else:
                prev_x, prev_y = 0, 0

            # Draw the hand annotations on the image
            mp_drawing.draw_landmarks(image,hand_landmarks, mp_hands.HAND_CONNECTIONS) #here it was (image,hand_landmarks,mp_...)

    # Overlay the canvas on the image
    image = cv2.addWeighted(image, 1, canvas, 0.5, 0) #here it was image and canvas interchanged

    # Display the image
    cv2.imshow('Hand Tracking', image) #here it was image
    if cv2.waitKey(5) & 0xFF == 27:
        cv2.imwrite("saved_image.png",canvas)
        break

# Release the webcam and close OpenCV windows
cap.release()
cv2.destroyAllWindows()